# Crime since 2001 in Chicago

* Data source: [data.gov](https://catalog.data.gov/dataset/crimes-2001-to-present-398a4)


* Describe the dataset:

    This dataset reflects reported incidents of crime that occurred in the City of Chicago from 2001 to August 9, 2018, minus the most recent seven days.

    Columns in the dataset: ['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type', 'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat', 'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate', 'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude', 'Location']


* Outline the types of insights you hope to gain from it:

    1. Most common type of crime each year and each date.
    1. Tracking crime frequency by geohash(7) per year.
    1. 


* Make hypotheses about what you might find:

    1. A lot of crimes that are categorized as [Battery](https://en.wikipedia.org/wiki/Battery_(crime) type during Black Friday.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import datetime
import time

In [22]:
%%time

df = spark.read.format('csv')\
    .option('header', 'true')\
    .load('hdfs://orion11:13030/crime-since-2001-chicago.csv')\
    .limit(10)
df.show(5)

+--------+-----------+--------------------+--------------------+----+-------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|IUCR| Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+----+-------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10000092|   HY189866|03/18/2015 07:44:...|     047XX W OHIO ST|041A|      BATTERY| AGGRAVATED: HANDGUN|              STREET| false|   false|1111

In [3]:
df.columns

['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']

In [21]:
serialize_date = udf(lambda d: datetime.strptime(d, '%m/%d/%Y %I:%M:%S %p'), DateType())

In [23]:
%%time

temp = df.withColumn('Date', serialize_date(df['Date']))

temp.show(3)

+-------+-----------+----------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|     ID|Case Number|      Date|               Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+-------+-----------+----------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|2097983|   HH321826|2002-04-21|     016XX E 70TH ST|0460|            BATTERY|              SIMPLE|           APARTMENT| false|    true|0332|     003|   5|     